In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime

from championWrapper import ChampionWrapper

In [2]:
in_data = pd.read_csv('pb_table.csv', index_col=False)
champs = ChampionWrapper()

num_champs = len(champs.champToID)
num_champs

/tmp/ipykernel_5904/3303039805.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  in_data = pd.read_csv('pb_table.csv', index_col=False)


158

In [3]:
in_data.head()

,Phase,Blue,Red,Score,Winner,Patch,BB1,RB1,BB2,RB2,...,BR3,BR4,BR5,RR1,RR2,RR3,RR4,RR5,SB,VOD
0,LCS/2021 Season/Lock In/Picks and Bans/Individ...,Team Liquid,Cloud9,3 - 2,1,11.2,Lulu,Pantheon,Senna,Renekton,...,Mid,Support,Top,Mid,Bot,Jungle,Support,Top,SB,https
1,LCS/2021 Season/Lock In/Picks and Bans/Individ...,Team Liquid,Cloud9,2 - 2,2,11.2,Lulu,Pantheon,Senna,Renekton,...,Support,Top,Mid,Bot,Mid,Jungle,Support,Top,SB,https
2,LCS/2021 Season/Lock In/Picks and Bans/Individ...,Cloud9,Team Liquid,1 - 2,1,11.2,Renekton,Pantheon,Camille,Lulu,...,Support,Mid,Top,Support,Jungle,Bot,Mid,Top,SB,https
3,LCS/2021 Season/Lock In/Picks and Bans/Individ...,Cloud9,Team Liquid,0 - 2,2,11.2,Renekton,Pantheon,Camille,Lulu,...,Support,Top,Mid,Jungle,Bot,Mid,Top,Support,SB,https
4,LCS/2021 Season/Lock In/Picks and Bans/Individ...,Team Liquid,Cloud9,1 - 0,1,11.2,Lulu,Pantheon,Lillia,Renekton,...,Mid,Bot,Support,Top,Mid,Jungle,Bot,Support,SB,https


In [4]:
#isolate the input frames of hero choices by name
isolated_input = in_data[['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5']]
isolated_input.head()

,BB1,RB1,BB2,RB2,BB3,RB3,BP1,RP1,RP2,BP2,BP3,RP3,RB4,BB4,RB5,BB5,RP4,BP4,BP5,RP5
0,Lulu,Pantheon,Senna,Renekton,Olaf,Camille,Udyr,Twisted Fate,Kai'Sa,Samira,Azir,Taliyah,Rell,Alistar,Gragas,Jayce,Leona,Nautilus,Gangplank,Irelia
1,Lulu,Pantheon,Senna,Renekton,Olaf,Camille,Taliyah,Kai'Sa,Twisted Fate,Samira,Rell,Udyr,Orianna,Alistar,Azir,Nautilus,Leona,Gnar,Syndra,Irelia
2,Renekton,Pantheon,Camille,Lulu,Udyr,Seraphine,Olaf,Thresh,Graves,Senna,Tahm Kench,Aphelios,Twisted Fate,Azir,Karma,Orianna,Syndra,Zoe,Malphite,Gangplank
3,Renekton,Pantheon,Camille,Lulu,Thresh,Twisted Fate,Olaf,Udyr,Kai'Sa,Senna,Tahm Kench,Azir,Orianna,Irelia,Syndra,Jayce,Gnar,Karma,LeBlanc,Blitzcrank
4,Lulu,Pantheon,Lillia,Renekton,Olaf,Rell,Taliyah,Camille,Twisted Fate,Jax,Galio,Udyr,Samira,Senna,Thresh,Alistar,Kai'Sa,Xayah,Leona,Sett


In [5]:
#Dataframe to Numpy
inputnpArray = isolated_input.to_numpy()
inputnpArray

numberpd = pd.DataFrame(inputnpArray, columns=['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5'])
print(numberpd)



array([['Lulu', 'Pantheon', 'Senna', ..., 'Nautilus', 'Gangplank',
        'Irelia'],
       ['Lulu', 'Pantheon', 'Senna', ..., 'Gnar', 'Syndra', 'Irelia'],
       ['Renekton', 'Pantheon', 'Camille', ..., 'Zoe', 'Malphite',
        'Gangplank'],
       ...,
       ['Renekton', 'Ziggs', 'Thresh', ..., 'Volibear', 'Rakan', 'Braum'],
       ['Ryze', 'LeBlanc', 'Tristana', ..., 'Sejuani', 'Jayce',
        'Cassiopeia'],
       ['Ryze', 'Kennen', 'Tristana', ..., 'Camille', 'Kindred', 'Jayce']],
      dtype=object)

In [6]:
# Numpy convert Names to Ids
for match in range(len(inputnpArray)):
    for column in range(len(inputnpArray[match])):
        inputnpArray[match][column] = champs.cNameToId(inputnpArray[match][column])



In [14]:
from WindowGenerator import WindowGenerator



WindowGenerator.WindowGenerator

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=num_champs,
        output_dim = 320
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
    tf.keras.layers.Dense(192, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1)
])
model.summary()

2022-03-22 13:46:42.110587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-22 13:46:42.110900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2e:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-22 13:46:42.115780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-22 13:46:42.115792: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 320)         50560     
                                                                 
 bidirectional (Bidirectiona  (None, 512)              1181696   
 l)                                                              
                                                                 
 dense (Dense)               (None, 192)               98496     
                                                                 
 dropout (Dropout)           (None, 192)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 193       
                                                                 
Total params: 1,330,945
Trainable params: 1,330,945
Non-trainable params: 0
______________________________________________

In [9]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

In [10]:
history = model.fit(inputnpArray, epochs=100)

Epoch 1/100


ValueError: in user code:

    File "/home/inzi/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/inzi/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/inzi/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/inzi/.local/lib/python3.8/site-packages/keras/engine/training.py", line 861, in train_step
        self._validate_target_and_loss(y, loss)
    File "/home/inzi/.local/lib/python3.8/site-packages/keras/engine/training.py", line 818, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<keras.losses.CategoricalCrossentropy object at 0x7f50cc0a0400>, and therefore expects target dat